In [2]:
import sys
import pandas as pd
import numpy as np
import re as regex
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from collections import Counter

# Loading of Raw Data

In [4]:
trainData = pd.read_csv('../data/data/train.csv', header = 0)
testData = pd.read_csv('../data/data/test.csv', header = 0)

print(trainData.shape)
print(testData.shape)

(95851, 8)
(226998, 2)


In [5]:
trainData.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,22256635,"Nonsense? kiss off, geek. what I said is true...",1,0,0,0,0,0
1,27450690,"""\n\n Please do not vandalize pages, as you di...",0,0,0,0,0,0
2,54037174,"""\n\n """"Points of interest"""" \n\nI removed the...",0,0,0,0,0,0
3,77493077,Asking some his nationality is a Racial offenc...,0,0,0,0,0,0
4,79357270,The reader here is not going by my say so for ...,0,0,0,0,0,0


In [6]:
testData.head()

,id,comment_text
0,6044863,==Orphaned non-free media (Image:41cD1jboEvL. ...
1,6102620,::Kentuckiana is colloquial. Even though the ...
2,14563293,"Hello fellow Wikipedians,\nI have just modifie..."
3,21086297,"AKC Suspensions \nThe Morning Call - Feb 24, 2..."
4,22982444,== [WIKI_LINK: Talk:Celts] ==


# Accessing elements (examples)

In [7]:
print(trainData.loc[trainData.loc[:, "toxic"] == 1,].shape[0])
print(trainData.loc[trainData.loc[:, "severe_toxic"] == 1,].shape[0])
print(trainData.loc[trainData.loc[:, "obscene"] == 1,].shape[0])
print(trainData.loc[trainData.loc[:, "threat"] == 1,].shape[0])
print(trainData.loc[trainData.loc[:, "insult"] == 1,].shape[0])
print(trainData.loc[trainData.loc[:, "identity_hate"] == 1,].shape[0])

9237
965
5109
305
4765
814


In [11]:
toxicData = trainData.loc[trainData.loc[:, "toxic"] == 1,]
severeToxicData = trainData.loc[trainData.loc[:, "severe_toxic"] == 1,]
obsceneData = trainData.loc[trainData.loc[:, "obscene"] == 1,]
threatData = trainData.loc[trainData.loc[:, "threat"] == 1,]
insultData = trainData.loc[trainData.loc[:, "insult"] == 1,]
identityHateData = trainData.loc[trainData.loc[:, "identity_hate"] == 1,]

# Create BoW

In [12]:
#vectorizer = TfidfVectorizer(max_df=1.0, min_df=0.001, stop_words = ENGLISH_STOP_WORDS, token_pattern=u'(?u)\\b\\w\\w+\\b')
vectorizer = TfidfVectorizer(max_df=1.0, min_df=0.001, stop_words = ENGLISH_STOP_WORDS, token_pattern=u'\\b[a-zA-Z]\w+\\b')

data_corpus = trainData.comment_text
X = vectorizer.fit_transform(data_corpus)
# get frequencies of words
freqs = [(word, X.getcol(idx).sum()) for word, idx in vectorizer.vocabulary_.items()]


In [26]:
vectorizer = TfidfVectorizer(max_df=0.2, min_df=0.01, stop_words = ENGLISH_STOP_WORDS, token_pattern=u'\\b[a-zA-Z]\w+\\b')
toxicData_corpus = toxicData.comment_text
X = vectorizer.fit_transform(data_corpus)
freqs = [(word, X.getcol(idx).sum()) for word, idx in vectorizer.vocabulary_.items()]

In [31]:
help(vectorizer)

Help on TfidfVectorizer in module sklearn.feature_extraction.text object:

class TfidfVectorizer(CountVectorizer)
 |  Convert a collection of raw documents to a matrix of TF-IDF features.
 |  
 |  Equivalent to CountVectorizer followed by TfidfTransformer.
 |  
 |  Read more in the :ref:`User Guide <text_feature_extraction>`.
 |  
 |  Parameters
 |  ----------
 |  input : string {'filename', 'file', 'content'}
 |      If 'filename', the sequence passed as an argument to fit is
 |      expected to be a list of filenames that need reading to fetch
 |      the raw content to analyze.
 |  
 |      If 'file', the sequence items must have a 'read' method (file-like
 |      object) that is called to fetch the bytes in memory.
 |  
 |      Otherwise the input is expected to be the sequence strings or
 |      bytes items are expected to be analyzed directly.
 |  
 |  encoding : string, 'utf-8' by default.
 |      If bytes or files are given to analyze, this encoding is used to
 |      decode.
 

In [27]:
# sort according to descending frequencies
freqs = sorted (freqs, key = lambda x: -x[1])

freqs[0:500]

[('talk', 3847.1784732587298),
 ('page', 3694.4734956941193),
 ('wikipedia', 3452.2645737099001),
 ('just', 2733.3305125751103),
 ('like', 2613.7208163312648),
 ('don', 2391.9419358211317),
 ('think', 2137.7945252462555),
 ('thanks', 2108.8683777129186),
 ('know', 2068.6405096617859),
 ('edit', 1911.8878067744085),
 ('people', 1800.8652179207747),
 ('did', 1800.840449898086),
 ('time', 1638.1105409711299),
 ('user', 1613.9778685218346),
 ('articles', 1538.818161248709),
 ('good', 1527.1192254604891),
 ('ve', 1510.0793580404272),
 ('use', 1469.804460955319),
 ('utc', 1438.1208967267376),
 ('make', 1378.5761906888872),
 ('does', 1354.2783952995787),
 ('thank', 1341.72242750073),
 ('want', 1303.5374474276689),
 ('wp', 1287.7830185191094),
 ('way', 1251.7280468032736),
 ('section', 1227.3384242287452),
 ('stop', 1219.229302864939),
 ('information', 1217.807944693225),
 ('new', 1213.784822537707),
 ('editing', 1212.6945328974928),
 ('say', 1202.0236007768406),
 ('sources', 1199.359953450869

In [23]:
# sort according to descending frequencies
freqs = sorted (freqs, key = lambda x: -x[1])

freqs[0:500]

[('article', 4321.8949476516764),
 ('talk', 3820.5104084668023),
 ('page', 3668.4520826854882),
 ('wikipedia', 3424.1755082641116),
 ('just', 2714.8821694465723),
 ('like', 2597.0727662851423),
 ('don', 2376.5895142112308),
 ('think', 2118.7716297940392),
 ('thanks', 2093.9664615955985),
 ('know', 2057.4652060076687),
 ('edit', 1900.3938575417055),
 ('did', 1789.6721646821111),
 ('people', 1788.7361713914952),
 ('time', 1626.6123629592287),
 ('user', 1606.7188489166583),
 ('articles', 1519.6724048061656),
 ('good', 1510.8332682582102),
 ('ve', 1499.9589891412586),
 ('use', 1461.6857807546983),
 ('utc', 1430.6549936765032),
 ('make', 1368.6261787584556),
 ('does', 1340.9671379148226),
 ('thank', 1334.5291603240833),
 ('want', 1294.5473379435036),
 ('wp', 1277.1351585728969),
 ('way', 1241.6865835226322),
 ('stop', 1214.2135395009948),
 ('section', 1212.7645095475239),
 ('editing', 1206.3453001535918),
 ('new', 1203.8662510946499),
 ('information', 1201.1335735859916),
 ('say', 1193.8878

## Cleaning: Removing URLs (All test codes below)

In [46]:
class TwitterCleanuper:

    def iterate(self):
        for cleanup_method in [self.remove_urls,
                               self.remove_usernames,
                               self.remove_na,
                               self.remove_special_chars,
                               self.remove_numbers]:
            yield cleanup_method

    @staticmethod
    def remove_by_regex(tweets, regexp):
        tweets.loc[:, "comment_text"].replace(regexp, "", inplace=True)
        return tweets

    def remove_urls(self, tweets):
        return TwitterCleanuper.remove_by_regex(tweets, regex.compile(r"http.?://[^\s]+[\s]?"))

    def remove_na(self, tweets):
        return tweets[tweets["comment_text"] != "Not Available"]

    def remove_special_chars(self, tweets):  # it unrolls the hashtags to normal words
        for remove in map(lambda r: regex.compile(regex.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$",
                                                                     "@", "%", "^", "*", "(", ")", "{", "}",
                                                                     "[", "]", "|", "/", "\\", ">", "<", "-",
                                                                     "!", "?", ".", "'",
                                                                     "--", "---", "#", "\n"]):
            tweets.loc[:, "comment_text"].replace(remove, "", inplace=True)
        return tweets

    def remove_usernames(self, tweets):
        return TwitterCleanuper.remove_by_regex(tweets, regex.compile(r"@[^\s]+[\s]?"))

    def remove_numbers(self, tweets):
        return TwitterCleanuper.remove_by_regex(tweets, regex.compile(r"\s?[0-9]+\.?[0-9]*"))

In [47]:
class Cleaner():
    
    def __init__(self):
        self.processed_data = 0
    
    def cleanup(self, data, cleanuper):
        t = data
        for cleanup_method in cleanuper.iterate():
            t = cleanup_method(t)
        self.processed_data = t
        
    def getData(self):
        return self.processed_data

In [48]:
cleanup = TwitterCleanuper()
cleaner = Cleaner()
cleaner.cleanup(trainData, cleanup)
trainData = cleaner.getData()
trainData.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,22256635,Nonsense kiss off geek what I said is true I...,1,0,0,0,0,0
1,27450690,Please do not vandalize pages as you did with...,0,0,0,0,0,0
2,54037174,Points of interest I removed the points of in...,0,0,0,0,0,0
3,77493077,Asking some his nationality is a Racial offenc...,0,0,0,0,0,0
4,79357270,The reader here is not going by my say so for ...,0,0,0,0,0,0


In [ ]:
class TwitterData_TokenStem():
    def __init__(self, data):
        self.processed_data = data
        
    def stem(self, stemmer=nltk.PorterStemmer()):
        def stem_and_join(row):
            row["comment_text"] = list(map(lambda str: stemmer.stem(str.lower()), row["comment_text"]))
            return row
        self.processed_data = self.processed_data.apply(stem_and_join, axis=1)

    def tokenize(self, tokenizer=nltk.word_tokenize):
        def tokenize_row(row):
            row["comment_text"] = tokenizer(row["comment_text"])
            row["tokenized_text"] = [] + row["comment_text"]
            return row
        self.processed_data = self.processed_data.apply(tokenize_row, axis=1)
        
    def getData(self):
        return self.processed_data

In [ ]:
stemmer = TwitterData_TokenStem(data)
stemmer.tokenize()
stemmer.stem()
data = stemmer.getData()
data.head()

In [ ]:
words = Counter()
for idx in data.index:
    words.update(data.loc[idx, "comment_text"])

words.most_common(5)

In [ ]:
nltk.download('stopwords')

In [ ]:
stopwords=nltk.corpus.stopwords.words("english")
whitelist = ["n't", "not"]
for idx, stop_word in enumerate(stopwords):
    if stop_word not in whitelist:
        del words[stop_word]
words.most_common(5)

In [1]:
import nltk

In [2]:
install nltk("word_list")

SyntaxError: invalid syntax (<ipython-input-2-65b37cac269a>, line 1)

In [7]:
from nltk.stem import WordNetLemmatizer

In [4]:
stemmer = PorterStemmer()

In [9]:
lemmatizer = WordNetLemmatizer()

In [24]:
lemmatizer.lemmatize("edit")

'edit'

In [11]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Matthew\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True